In [ ]:
!git clone https://github.com/NVlabs/MambaVision.git

In [4]:
!git clone https://github.com/jiaowoguanren0615/MambaVision.git

Cloning into 'MambaVision'...
remote: Enumerating objects: 91, done.
remote: Counting objects: 100% (91/91), done.
remote: Compressing objects: 100% (69/69), done.
remote: Total 91 (delta 37), reused 71 (delta 21), pack-reused 0 (from 0)
Unpacking objects: 100% (91/91), 309.69 KiB | 1.97 MiB/s, done.


In [1]:
!pip install gdown

In [3]:
!pip install -r requirements.txt

  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.4/85.4 kB 963.3 kB/s eta 0:00:00 0:00:01
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.2/43.2 kB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 209.4/209.4 MB 6.6 MB/s eta 0:00:00:00:0100:01
  Created wheel for causal-conv1d: filename=causal_conv1d-1.4.0-cp310-cp310-linux_x86_64.whl size=104879097 sha256=bbeba7907618a3ba3342742c7a02c9dfd5c229f4bfb4026dc2e936148354d19d
  Stored in directory: /root/.cache/pip/wheels/e3/dd/4c/205f24e151736bd22f5980738dd10a19af6f093b6f4dcab006
  Created wheel for mamba-ssm: filename=mamba_ssm-2.2.2-cp310-cp310-linux_x86_64.whl size=323998290 sha256=a658a5438dbe9fb3a53799d7d9f4714ca09225769c24ec04a403728ac7d1c69e
  Stored in directory: /root/.cache/pip/wheels/57/7c/90/9f963468ecc3791e36e388f9e7b4a4e1e3f90fbb340055aa4d
Successfully built causal-conv1d mamba-ssm


In [23]:
import gdown

url = "https://drive.google.com/file/d/1MPX6vW_XiepXkbeM0LjQQasMxVSjLmEh/view?usp=sharing"

mambavision_tiny = "https://drive.google.com/file/d/1rTTgkw5b7VNaiX3HdtBd2RQojRLOqYom/view?usp=sharing"

output = "/kaggle/working/ozon_vision/MambaVision-T2-1K/config.json"

output1 = "mambavision_tiny2_1k.pth.tar"

gdown.download(url, output, quiet=False, fuzzy=True)

# gdown.download(mambavision_tiny, output1, quiet=False, fuzzy=True)

Downloading...
From: https://drive.google.com/uc?id=1MPX6vW_XiepXkbeM0LjQQasMxVSjLmEh
To: /kaggle/working/ozon_vision/MambaVision-T2-1K/config.json
100%|██████████| 854/854 [00:00<00:00, 2.89MB/s]


'/kaggle/working/ozon_vision/MambaVision-T2-1K/config.json'

In [ ]:
import zipfile
zip_ref = zipfile.ZipFile("/kaggle/working/dataset.zip", 'r')
zip_ref.extractall("/kaggle/working/")
zip_ref.close()

In [4]:
import os
import random
import numpy as np
import shutil
import torch

def clear_folder(folder):
    for filename in os.listdir(folder):
        file_path = os.path.join(folder, filename)
        try:
            if os.path.isfile(file_path) or os.path.islink(file_path):
                os.unlink(file_path)
            elif os.path.isdir(file_path):
                shutil.rmtree(file_path)
        except Exception as e:
            print('Failed to delete %s. Reason: %s' % (file_path, e))

def remove(path):
    os.remove(path)
    
# make archieve
def make_zip(
        output_filename = "/kaggle/working/images28",
        dir_name = "/kaggle/working/images"
):
    shutil.make_archive(output_filename, 'zip', dir_name)
    
def seed_everything(seed=42):
    os.environ['PYTHONHASHSEED'] = str(seed)
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

In [33]:
make_zip("/kaggle/working/output/r1_mamba224/models", "/kaggle/working/output/r1_mamba224/models")

In [ ]:
remove("/kaggle/working/mambavision_tiny2_1k.pth.tar")

In [ ]:
clear_folder("/kaggle/working/train")

In [ ]:
!torchrun --nproc_per_node=2 train.py --model_name "/kaggle/working/ozon_vision/MambaVision-T2-1K" \
--root_train /kaggle/working/train \
--root_test /kaggle/working/test \
--output_path /kaggle/working/output \
--name "r1_mamba224" \
--num_epochs 7 \
--batch_size 96 \
--im_size 224 \
--lr 5e-03 

In [ ]:
# !pip install torch==2.0.1 torchvision==0.15.2 torchaudio==2.0.2


In [ ]:
!python -m torch.distributed.run --nproc_per_node=2 train_gpu.py --data_root /kaggle/working/train --finetune /kaggle/working/ozon_vision/MambaVision-T2-1K --freeze_layers

In [ ]:
!git clone https://github.com/m-a-bot/ozon_vision.git